In [1]:
# This notebook checks is trying different pipeline approaches

# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
import pyprep as pp


In [2]:

#from Functions.main_meg_qc import initial_stuff
from Functions. data_load_and_folders import load_meg_data
duration=15
#n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration)

raw_file = './data/sub_HT05ND16/210811/mikado-1.fif/'
raw, mags, grads = load_meg_data(raw_file)

#crop the data to calculate faster
raw_cropped = raw.copy()
raw_cropped.crop(0, duration*60) 



Opening raw data file ./data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 3

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:8: RuntimeWarning: This filename (./data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_file)


<Raw | mikado-1.fif, 319 x 900001 (900.0 s), ~6.6 MB, data not loaded>

In [3]:
raw_new=raw.copy()

types1=raw_new.pick_types(meg=True)
types1

<Raw | mikado-1.fif, 306 x 3390000 (3390.0 s), ~6.6 MB, data not loaded>

In [4]:
# https://pyprep.readthedocs.io/en/latest/generated/pyprep.NoisyChannels.html#pyprep.NoisyChannels

noisy = pp.NoisyChannels(raw)

# Doesnt work. Only EEG is supported. Tried to change channel type in just to try in:
# opt/anaconda3/envs/mne_new/lib/python3.9/site-packages/pyprep/find_noisy_channels.py
# to self.raw_mne.pick_types(meg=True, verbose=True)
# no luck, it gives traceback on this line. 
# (If u explicitely call raw.pick_types(meg=True, verbose=True) - everything works fine).
# Dont know where the issue is exactly, but in the end we d need to rewrite this whole function 
# to work with MEG if we really need it.
#



ValueError: No channels match the selection.

In [5]:
# Annotate bad peak to peak amplitudes: peaks and flats

# https://mne.tools/stable/generated/mne.preprocessing.annotate_amplitude.html
# copmpare this MNE function with what I wrote myself: Funnks - > Peaks_meg_qc -> neighbour_peak_amplitude

#'Creates annotations BAD_peak or BAD_flat for spans of data where consecutive samples exceed the threshold in peak or fall below the threshold 
# in flat for more than min_duration' 
# min_duration is in minutes
#peaks abd flats are in the same scale as the data, like: x^-13

# Differences with mine ACCORDING TO DOCUNMENTATION: 
# - no option to set distance between up and down peak to be concedered a pair. Maybe they calculate it differently? check code.
# - This function need raw obj as input. So not epochs and not just a piece of data -> need to adjust it for epochs. 
# - Can set here extra: flat, percent of bad, min duration and actual heigh of peaks and flats.

amplit_annot=mne.preprocessing.annotate_amplitude(raw_cropped, peak=3e-11, flat=5e-15, bad_percent=5, min_duration=0.005, verbose=True)

raw_cropped.set_annotations(raw_cropped.annotations + amplit_annot[0])  
# For some reason annotate_amplitude creates a tuple, not just annotations. 
# This whole tuple cant be added to annots, need to slice the first part like: [0]

raw_cropped.plot() #plot the data with annotations


Finding segments below or above PTP threshold.


2022-07-14 13:36:28,471 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


Using qt as 2D backend.
Opening raw-browser...
Using pyopengl with version 3.1.1a1


2022-07-14 13:36:29,531 - OpenGL.acceleratesupport - INFO - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [ ]:
# Annotate breaks

break_annots = mne.preprocessing.annotate_break(
    raw=raw_cropped,
    min_break_duration=20,  # consider segments of at least 20 s duration
    t_start_after_previous=5,  # start annotation 5 s after end of previous one
    t_stop_before_next=2  # stop annotation 2 s before beginning of next one
)

raw_cropped.set_annotations(raw_cropped.annotations + break_annots)  # add to existing
raw_cropped.plot()

In [ ]:
# Annotate movement

mne.preprocessing.annotate_movement